In [1]:
import pandas as pd
import numpy as np
import pickle
from itertools import chain
from collections import Counter
import operator

In [2]:
data_dir = '../prd_review_v2/data/'

prev_dt = '190104'
new_dt = '190117'

In [3]:
def get_df(dt):
    df_result = pd.read_pickle(data_dir+'df_result_'+dt+'.pkl')
    df_test = pd.read_pickle(data_dir+'df_test_'+dt+'.pkl')

    with open(data_dir+'word_index_'+dt+'.pkl', 'rb') as pkl_file:
        vocab_dict = pickle.load(pkl_file)            
    vocab_list = [x for x in range(len(vocab_dict)+1)]
    for k, v in vocab_dict.items():
        vocab_list[v] = k
        
    df_test = df_test.head(len(df_result)).reset_index(drop=True)
    # df_test.x = df_test.x.apply(lambda x: x.split())
    df_test = df_test.rename({'x': 'origin_tokens'}, axis=1)
    
    assert len(pd.Series(df_result.y == df_test.y).unique()) == 1
    
    df = pd.concat([df_result, df_test.drop(['y'], axis=1)], axis=1)
    df['tokens'] = df.x.apply(lambda x_list: [vocab_list[x] for x in x_list])
    
    return df

In [4]:
df_prev = get_df(prev_dt)
df_prev.sample(10)

,x,y,alpha,pred,m_id,text,origin_tokens,tokens
37467,"[16, 913, 15, 346, 2, 6, 450, 16, 181, 57, 23,...",0,"[3.3648682e-06, 4.649438e-06, 7.524111e-06, 2....",0,135414876,배송완료로 되어 확인 해보니 엉뚱한 곳에 배송해놓고 상품제고가 없다고 하며 다시 배...,"[배송, 완료, 되, 확인, 하, 보, 엉뚱, 곳, 배송, 놓, 상품, 제고, 없,...","[배송, 완료, 되, 확인, 하, 보, 곳, 배송, 놓, 상품, 없, 하, 다시, ..."
32500,"[1593, 7, 508, 31, 881, 1593, 2, 10, 413, 425,...",1,"[4.1742005e-05, 7.212325e-05, 8.5976266e-05, 3...",1,134824992,박시하게 입으려고 주문했는데 그렇게 박시 하지는 않네요 어깨라인이 예쁘고 가지색 다...,"[박시, 입, 77, 주문, 그렇게, 박시, 하, 않, 어깨, 라인, 예쁘, 4, ...","[박시, 입, 77, 주문, 그렇게, 박시, 하, 않, 어깨, 라인, 예쁘, 4, ..."
15929,"[20, 94, 11, 99, 1294, 37, 6, 761, 41, 2888, 4...",1,"[5.0232196e-05, 4.1103318e-05, 3.672666e-05, 2...",1,134660354,크게 기대하고 구매하진 안았은데 막상 받아보니 부들부들한 촉 감과 롱한 길이감이 좋...,"[크, 기대, 구매, 알, 막상, 받, 보, 부들부들, 한, 촉, 감과, 롱, 길이...","[크, 기대, 구매, 알, 막상, 받, 보, 부들부들, 한, 촉, 롱, 길이, 좋,..."
37559,"[1011, 33, 20, 3, 47, 41, 998, 243, 65, 21, 81...",1,"[3.0465697e-05, 5.817652e-06, 4.7665176e-06, 9...",1,135295556,지난번 큰거사고 한자리 남아서 작은사이즈 하나더 추가했어요 하부수납 엄청 많이되구요...,"[지난, 번, 크, 것, 살, 한, 자리, 남, 작, 사이즈, 하나, 더, 추가, ...","[지난, 번, 크, 것, 살, 한, 자리, 남, 작, 사이즈, 하나, 더, 추가, ..."
10539,"[208, 44, 66, 3181, 3181, 100, 99, 878, 41, 59...",0,"[9.831535e-05, 1.1463505e-05, 1.0596146e-05, 1...",1,135529347,블랙으로 구입 정말 so so 전 알바용 한 해 막 있을거라 딱 그 용도이고 무겁지...,"[블랙, 구입, 정말, so, so, 전, 알, 바, 한, 해, 막, 있, 것, 딱...","[블랙, 구입, 정말, so, so, 전, 알, 바, 한, 해, 막, 있, 것, 딱..."
18179,"[1057, 103, 24, 182, 79, 546, 558, 546, 206, 1...",1,"[1.0976492e-05, 1.7884242e-05, 7.733629e-06, 3...",1,135372562,소문듣고 샀는데 진짜 촉촉하고 보습까지 수분 보습이 같이되는게 신기해요 또살거에여,"[소문, 들, 사, 진짜, 촉촉, 보습, 수분, 보습, 같이, 되, 것, 신기, 또...","[소문, 들, 사, 진짜, 촉촉, 보습, 수분, 보습, 같이, 되, 것, 신기, 또..."
13355,"[747, 1024, 134, 13, 545, 60, 0, 0, 0, 0, 0, 0...",1,"[0.00020830633, 0.000103304155, 3.6860307e-05,...",1,134960501,친정부모님집 안방에 딱이네요,"[친정, 부모, 집, 안, 방, 딱]","[친정, 부모, 집, 안, 방, 딱, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
35914,"[348, 3517, 5, 126, 61, 1804, 0, 0, 0, 0, 0, 0...",1,"[1.7150964e-05, 1.0782319e-05, 1.7354621e-05, ...",0,134949859,왜앙고라인지잘모르겠고그냥평범합니다,"[왜, 앙고라, 잘, 모르, 그냥, 평범]","[왜, 앙고라, 잘, 모르, 그냥, 평범, 0, 0, 0, 0, 0, 0, 0, 0..."
8099,"[1, 19, 52, 44, 353, 12, 19, 0, 0, 0, 0, 0, 0,...",1,"[4.90293e-05, 0.00012398463, 3.345584e-05, 1.3...",1,134695122,좋은제품 저렴하게 구입했어요 꾸준히 사용하던 제품입니다,"[좋, 제품, 저렴, 구입, 꾸준히, 사용, 제품]","[좋, 제품, 저렴, 구입, 꾸준히, 사용, 제품, 0, 0, 0, 0, 0, 0,..."
2797,"[5, 37, 77, 372, 1, 50, 3, 212, 85, 76, 161, 1...",0,"[6.874996e-05, 3.9390234e-05, 4.6739293e-05, 5...",1,134793459,잘받았습니다 부드러운 촉감은좋은데 생각했던것보다 재질이얇네요 조금 아쉽긴하지만 잘 ...,"[잘, 받, 부드럽, 촉감, 좋, 생각, 것, 재질, 얇, 조금, 아쉽, 않, 잘, 입]","[잘, 받, 부드럽, 촉감, 좋, 생각, 것, 재질, 얇, 조금, 아쉽, 않, 잘,..."


In [5]:
df_new = get_df(new_dt)
df_new.sample(10)

,x,y,alpha,pred,m_id,text,origin_tokens,tokens
31324,"[1232, 399, 597, 194, 37, 980, 388, 1, 5, 119,...",1,"[6.939214e-05, 3.0142899e-05, 4.824522e-05, 2....",1,135526101,지난 월방송때 사고 지인이 좋다하여 개 더 구매했어요 기장도 좋고 가격대비 따뜻하고...,"[지난, 10, 월, 방송, 때, 사고, 지인, 좋, 하, 2, 개, 더, 구매, ...","[지난, 10, 월, 방송, 때, 사고, 지인, 좋, 하, 2, 개, 더, 구매, ..."
31234,"[399, 32, 1287, 3, 42, 40, 0, 0, 0, 0, 0, 0, 0...",1,"[0.000642442, 0.00047151433, 0.00082602893, 0....",1,134757016,살 여아 잘맞고 이쁘네요,"[10, 살, 여아, 잘, 맞, 이쁘]","[10, 살, 여아, 잘, 맞, 이쁘, 0, 0, 0, 0, 0, 0, 0, 0, ..."
32951,"[1827, 278, 25, 23, 10, 2, 327, 181, 3618, 5, ...",1,"[0.061076973, 0.032055598, 0.022672543, 0.0158...",1,135291327,이데베논을 여러번 주문해서 쓰고 있는데요 첨에는 얼굴이 화해진다고 하나요 열기운이 ...,"[이데베논, 여러, 번, 주문, 쓸, 있, 첨, 얼굴, 화해, 진다고, 하, 열, ...","[이데베논, 여러, 번, 주문, 쓸, 있, 첨, 얼굴, 진다고, 하, 열, 기운, ..."
14007,"[26, 51, 99, 8, 367, 72, 4, 47, 3, 4, 0, 0, 0,...",1,"[0.00071205175, 0.0006358825, 0.00023226932, 0...",1,134872632,빙금 받았습니다 색상도 마음에 들고 촉감도 부드럽고 입으면 편하겠어요 잘 입을께요,"[빙금, 받, 색상, 마음, 들, 촉감, 부드럽, 입, 편하, 잘, 입]","[받, 색상, 마음, 들, 촉감, 부드럽, 입, 편하, 잘, 입, 0, 0, 0, ..."
5349,"[416, 22, 577, 9, 711, 284, 0, 0, 0, 0, 0, 0, ...",1,"[0.0011259639, 0.00049403694, 0.0007568611, 0....",1,135700244,할인도 되고 적립금 사용해서 싸게 샀어요,"[할인, 되, 적립금, 사용, 쌓, 샀어요]","[할인, 되, 적립금, 사용, 쌓, 샀어요, 0, 0, 0, 0, 0, 0, 0, ..."
33791,"[9, 47, 18, 469, 12, 134, 76, 16, 82, 1, 0, 0,...",1,"[0.0014608497, 0.0013840487, 0.001019698, 0.00...",1,134701090,사용하기도 편하고 제품도 나쁘지않아 재구매 합니다 배송도 빠르고 좋아요,"[사용, 편하, 제품, 나쁘, 않, 재구매, 합니다, 배송, 빠르, 좋]","[사용, 편하, 제품, 나쁘, 않, 재구매, 합니다, 배송, 빠르, 좋, 0, 0,..."
21965,"[194, 23, 26, 31, 11, 90, 783, 79, 0, 0, 0, 0,...",1,"[0.0017377147, 0.00066345424, 0.0010065128, 0....",1,134772771,방송보다 주문해받아 써보니 촉촉하고 윤기 나는 듯해야,"[방송, 주문, 받, 쓰, 보, 촉촉, 윤기, 날]","[방송, 주문, 받, 쓰, 보, 촉촉, 윤기, 날, 0, 0, 0, 0, 0, 0,..."
47405,"[1, 5, 332, 142, 12, 13, 478, 7, 310, 12, 196,...",1,"[5.1001098e-05, 9.089592e-05, 3.7177266e-05, 4...",1,135921257,좋아하는 스타일이라 고민하지 않고 구매했어요 몸에 너무 붙지않을까 걱정했는데 반 착...,"[좋, 하, 스타일, 고민, 않, 구매, 몸, 너무, 붙, 않, 걱정, 55, 반,...","[좋, 하, 스타일, 고민, 않, 구매, 몸, 너무, 붙, 않, 걱정, 55, 반,..."
27023,"[274, 51, 349, 62, 4976, 237, 35, 462, 1329, 2...",0,"[4.962769e-05, 5.256291e-05, 0.00011064964, 0....",0,134716749,화면보다 색상이밝고 겨울이라 수면양말에 착용할려고 한치수위로 주문했으나 볼부분과 신...,"[화면, 색상, 밝, 겨울, 수면양말, 착용, 한, 치, 수위, 250, 주문, 볼...","[화면, 색상, 밝, 겨울, 수면양말, 착용, 한, 치, 250, 주문, 볼, 부분..."
13012,"[147, 126, 10, 990, 990, 118, 10, 2145, 246, 7...",1,"[8.7045395e-05, 2.5466852e-05, 1.2602258e-05, ...",1,135342753,역시 믿고 쓰는 에이지투웨니스 에이지투웨니스 팩트 쓰다가 마치 내가 피부가 좋아진것...,"[역시, 믿, 쓸, 에이지투웨니스, 에이지투웨니스, 팩트, 쓸, 마치, 내, 피부,...","[역시, 믿, 쓸, 에이지투웨니스, 에이지투웨니스, 팩트, 쓸, 마치, 내, 피부,..."


In [6]:
# attention model 내에서 alpha값이 큰 순서대로 index 정렬
# "UNKNOWN" seq == 0 최초로 "UNKNOWN"이후 토큰들은 모두 "UNKNOWN"이므로 고려하지 않음
# (고려되는 토큰 길이 / 3) + 1개 고려
def get_max_args(row):
    alpha = np.array(row['alpha'])
    x =  np.array(row['x'])
    zero_index = np.where(x == 0)[0].tolist()
    if zero_index:
        sent_size = int(zero_index[0])
    else:
        sent_size = int(len(x))
    num_alpha = int(sent_size / 3) + 1
    alpha = alpha[:sent_size].argsort()[-num_alpha:][::-1]
    return alpha.tolist()

In [7]:
df_prev['max_args'] = df_prev.apply(lambda row: get_max_args(row), axis=1)
df_new['max_args'] = df_new.apply(lambda row: get_max_args(row), axis=1)

In [8]:
# 위에서 산출한 token idx를 token으로변환
def get_alpha_tokens(row):
    max_args = row['max_args']
    xs = row['x']
    tokens = row['tokens']
    alpha_tokens = list()
    try:
        alpha_tokens = [tokens[arg] for arg in max_args if xs[arg] != 0]
    except IndexError:
        print("IndexError")
        alpha_tokens = None
    finally:
        return alpha_tokens

In [9]:
df_prev['alpha_tokens'] = df_prev.apply(lambda row: get_alpha_tokens(row), axis=1)
df_new['alpha_tokens'] = df_new.apply(lambda row: get_alpha_tokens(row), axis=1)

In [10]:
df_new.loc[(df_new.y == 0) & (df_new.pred == 0), ['text', 'alpha_tokens', 'tokens']].sample(10)

,text,alpha_tokens,tokens
18876,옷이 가격표도 없고 아마 누군가 한 주입고 반품한것인가하는 의문이 드네요 목에 때가...,"[불편, 조금, 지퍼, 아이, 들, 실망, 사용, 발송, 의문, 선택, 짓]","[옷, 가격표, 표, 없, 아마, 누군가, 한, 2, 주, 입, 반품, 하, 의문,..."
38971,색깔도 별로구요 스판끼도 없고 길이도 길고 대략난감입니다,"[대략, 난감, 별로, 구요]","[색깔, 별로, 구요, 스판, 끼, 없, 길이, 길, 대략, 난감, 0, 0, 0,..."
49089,봉제가 너무 허술해서 재봉재 맡겨야겠네요 두껍고 한철 입을만은해요,"[해요, 입, 철, 만]","[봉제, 너무, 허술, 봉재, 맡기, 두껍, 한철, 철, 입, 만, 해요, 0, 0..."
27490,악세서리 보관함으로는 매우 불편하네요 칸칸이가 부실하고 높이는 높고 손으로 보관된 ...,"[저렴, 가격, 그만큼, 물론, 후회, 괜히, 불편, 샀]","[보관함, 매우, 불편, 칸, 칸, 이, 부실, 높이, 높, 손, 보관, 늘, 빼,..."
42061,방금받았네요아무리택배파업이라하더라도문자라도주시던지 일이나걸렸네요,"[문자, 주, 하, 일]","[방금, 받, 아무리, 택배, 파업, 하, 문자, 주, 10, 일, 걸리, 0, 0..."
34511,품질은 고장없이 좀 오래 사용되면 좋음거니 지금 평할 수는 없음 배송이 너무 느려서...,"[소비자, 자, 피해, 부족, 보, 대처, 계속, 팔, 물건]","[품질, 고장, 없이, 좀, 오래, 사용, 좋, 지금, 없, 배송, 너무, 느리, ..."
37086,a급 볼이라 했는데 c 급도 안되는 제품같다 너무 닳았다,"[닳, a, 같, 했, 안]","[a, 급, 볼, 했, c, 급, 안, 되, 제품, 같, 너무, 닳, 0, 0, 0..."
23504,다른옷들에비해 조금 타이트한거같아 한칫수큰걸로 교환했네요 안에이너웨어입고입기엔 넉넉...,"[한, 같, 안, 큰, 교환, 타이트, 칫수, 부분, 이너, 크, 조금]","[다른, 옷, 들, 비해, 조금, 타이트, 같, 한, 칫수, 큰, 교환, 안, 이너..."
35188,세가지 색상중 네이비가 골반 부위가 더 타이트하고 그래서인지 허리부블도 다른 두 색...,"[그냥, 입, 들, 있, 상품, 반품, 그게, 귀찮, 묻, 많이, 두]","[세, 가지, 색상, 네이비, 골반, 부위, 더, 타이트, 그렇, 허리, 부, 블,..."
48638,배송된제품봤는데 이렇게왔네요 ㅠ 반품도교환도 귀찮아서그냥씁니다만 이러심않됩니다,"[그냥, 이러, 않, 귀찮]","[배송, 제품, 봤, 이렇게, 왔, 반품, 교환, 귀찮, 그냥, 이러, 않, 0, ..."


In [11]:
df_new.loc[(df_new.y == 1) & (df_new.pred == 1), ['text', 'alpha_tokens', 'tokens']].sample(10)

,text,alpha_tokens,tokens
3751,털빠짐 말고는 진짜 다 좋아요 근데 털이 너무너무 많이 빠져서ㅠ,"[빠지, 털, 맡, 진짜]","[털빠짐, 맡, 진짜, 다, 좋, 근데, 털, 너무너무, 많이, 빠지, 0, 0, ..."
39654,배송은 일주일정도 길게 걸렸구요 상품자체로는 나무랄곳없네요 편하고 좋아용,"[정도, 나무랄, 길, 자체]","[배송, 일주일, 정도, 길, 걸리, 상품, 자체, 나무랄, 없, 편하, 좋, 0,..."
46997,아직 써보진 않았지만 넘 맘에 들어요 수납도 많이 되요,"[많이, 되, 수납, 들]","[아직, 쓰, 보, 않, 넘, 맘, 들, 수납, 많이, 되, 0, 0, 0, 0, ..."
45349,세 킬로 아들 넉넉하게 잘 맞아요 빨아도 늘어짐 없이 좋네요,"[좋, 빨, 없이, 늘어짐, 맞]","[7, 세, 21, 킬로, 아들, 넉넉, 잘, 맞, 빨, 늘어짐, 없이, 좋, 0,..."
50048,박스에손상없이 잘도착했네요 매수도많고 오래쓸거겉아요 일반티슈한장은 한번쓰고버리기 큰...,"[쓸, 티슈, 딱, 큰, 미니, 버리, 일반, 한]","[박스, 손상, 없이, 잘, 도착, 많, 오래, 쓸, 겉, 일반, 티슈, 한, 장,..."
761,항상구매하여쓰는제품입니다 발진도 없고좋아요,"[항상, 좋, 구매]","[항상, 구매, 쓸, 제품, 발진, 없, 좋, 0, 0, 0, 0, 0, 0, 0,..."
48064,항상 먹고 좋아하는 간식인데 한동안 간식 걱정은 안 해도 되겠네요 다 먹고나면 재구...,"[다, 나, 먹, 재구매, 해도, 되]","[항상, 먹, 좋, 하, 간식, 한, 동안, 간식, 걱정, 안, 해도, 되, 다, ..."
31377,블루종 네이비 구입했다가 디자인도 이쁘고 저럼해서 아이보리하나더 구입했어요 이쁘게 ...,"[입힐, 잘, 아이보리, 네이비, 디자인]","[네이비, 구입, 디자인, 이쁘, 저, 럼, 아이보리, 하나, 더, 구입, 이쁘, ..."
50241,언제나 믿고 쓰는 에이지투웨니스 팩트 ㅋ 다 떨어져서 다른 거 일주일 쓰다가 도저히...,"[일주일, 쓸, 도저히, 떨어지, 안, 다른]","[언제나, 믿, 쓸, 에이지투웨니스, 팩트, ㅋㅋ, ㅋㅋ, 다, 떨어지, 다른, 일..."
24088,좋아요 크림통꺼는좀되서바르기가뻑뻑해요 잘펴바르기가불편합니다 소프트하고바름이좀좋앗으면합니다,"[좀, 좋, 합니다, 불편, 소프트, 바름, 바르]","[좋, 크림, 통, 꺾, 좀, 되, 바르, 뻑뻑, 해요, 잘, 펴, 바르, 불편, ..."


In [12]:
df_prev['correct'] = (df_prev.y == df_prev.pred)
df_new['correct'] = (df_new.y == df_new.pred)

In [13]:
def get_accuarcy(df):
    return len(df.loc[(df.correct == True)]) / len(df)


def get_f1_score(df):
    tp = len(df.loc[(df.y == 1) & (df.pred == 1)])
    fp = len(df.loc[(df.y == 0) & (df.pred == 1)])
    tn = len(df.loc[(df.y == 1) & (df.pred == 0)])
    fn = len(df.loc[(df.y == 0) & (df.pred == 0)])
    
    pr_0 = fn / (fp+fn)
    rc_0 = fn / (tn+fn)
    f1_0 = 2*pr_0*rc_0/(pr_0+rc_0)

    pr_1 = tp / (tp+tn)
    rc_1 = tp / (tp+fp)
    f1_1 = 2*pr_1*rc_1/(pr_1+rc_1)

    return f1_0, f1_1

In [14]:
acc_prev = get_accuarcy(df_prev)
acc_new = get_accuarcy(df_new)
print("acc_prev : {}, acc_new: {}".format(acc_prev, acc_new))

acc_prev : 0.852729301948052, acc_new: 0.860129176980198


In [15]:
f1_neg_prev, f1_pos_prev = get_f1_score(df_prev)
f1_neg_new, f1_pos_new = get_f1_score(df_new)
print("f1_neg_prev : {}, f1_neg_new: {}".format(f1_neg_prev, f1_neg_new))
print("f1_pos_prev : {}, f1_pos_new: {}".format(f1_pos_prev, f1_pos_new))

f1_neg_prev : 0.6666666666666667, f1_neg_new: 0.6945136630485282
f1_pos_prev : 0.9054859189321179, f1_pos_new: 0.9093006633478375


In [16]:
alpha_tokens = df_new.loc[(df_new.correct == True) & (df_new.y == 1)].alpha_tokens.tolist()
counter = Counter(chain(*alpha_tokens))
alpha_count = sorted(counter.items(), key=operator.itemgetter(1))
alpha_count.reverse()